In [ ]:
# example_jax.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to use the jax options and some advantages
# Note only functions are returned when using jax!

In [ ]:
# Import generic stuff
import numpy as np
import matplotlib.pyplot as plt
import sys
from jax import grad

In [ ]:
# Font style
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

In [ ]:
# Import package
# Adding path to module
sys.path.append("../")
# Module imports
# TODO: Fix this
from fennel import Fennel, config
# picture path
PICS = '../pics/'

In [ ]:
# Plotting standards
std_size = 6.  # Size of the plot
fontsize = 20.  # Fontsize in the plot
lw=1.  # Linewidth
h_length=1  # Handle length for the legends
export_dpi = 500  # Dpi for the image export

In [ ]:
# Setting config
config["general"]["jax"] = True
config["general"]["random state seed"] = 42

In [ ]:
# Creating a fennel object
fennel_instance = Fennel()

In [ ]:
# Energies of interest
egrid = config['advanced']["energy grid"]
track_lengths = config['advanced']['track length']
wavelengths = config['advanced']['wavelengths']
angle_grid = config['advanced']['angles']
zgrid = config['advanced']['z grid']
n = config["mediums"][config["scenario"]["medium"]]["refractive index"]

In [ ]:
test_energy = 1e4

In [ ]:
# Track example
track_dcounts, track_angles = (
    fennel_instance.track_yields(
        test_energy,
        wavelengths=wavelengths,
        angle_grid=angle_grid,
        n=n,
        interaction='total',
        function=True
    )
)
# Using the internal value fetcher
track_dcounts_internal, track_angles_internal = (
    fennel_instance.track_yields(
        test_energy,
        wavelengths=wavelengths,
        angle_grid=angle_grid,
        n=n,
        interaction='brems',
        function=False
    )
)

In [ ]:
# Evalulating the fetched functions
# In jax mode only floats are evaluated
# Wrap the function in a custom function or list comprehension for
# multiple values
track_dcounts_val = np.array([
    track_dcounts(test_energy, wavelength)
    for wavelength in wavelengths
])
track_angles_val = np.array([
    track_angles(angle, n, test_energy)
    for angle in angle_grid
])

In [ ]:
# How to calculate the total counts
print(np.trapz(track_dcounts_val, wavelengths))
print(np.trapz(track_dcounts_internal, wavelengths))

In [ ]:
# Plotting some track stuff
# Photon counts
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(wavelengths, track_dcounts_val, linewidth=lw, color='k', ls='-')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Wavelength\;[nm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Differential\;Counts$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;Muon$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_Track_Wavelength_JAX.png",
               bbox_inches='tight', dpi=export_dpi)
# ----------------------------------------------
# ----------------------------------------------
# Angles
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(angle_grid, track_angles_val / np.trapz(track_angles_val, x=angle_grid),
         linewidth=lw, color='k', ls='-')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Emission\;Angle$', fontsize=fontsize)
ax1.set_ylabel(r'$PDF$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;Muon$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_Track_Angle_JAX.png",
               bbox_inches='tight', dpi=export_dpi)
# ---------------------------------------------
# ---------------------------------------------
# Differential function
grad_track_dcounts = grad(track_dcounts)
# Plot
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(wavelengths,
         np.array([
             grad_track_dcounts(test_energy, wavelength)
             for wavelength in wavelengths
         ]),
         linewidth=lw, color='k')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Wavelength\;[nm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Grad(Differential\;Counts)$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;Muon$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Grad_Counts_JAX.png",
               bbox_inches='tight', dpi=export_dpi)

In [ ]:
# Electromagnetic Cascade
em_dcounts, em_dcounts_sampler, em_long, em_angles = (
    fennel_instance.em_yields(
        energy=test_energy,
        particle=11,
        function=True
    )
)
# Using the internal value fetcher as a comparison
em_dcounts_internal, em_dcounts_sampler_internal, em_long_internal, em_angles_internal = (
    fennel_instance.em_yields(
        energy=test_energy,
        particle=-11,
        function=False
    )
)

In [ ]:
# Wrap in list comprehension for multiple values
em_dcounts_val = np.array([
    em_dcounts(test_energy, wavelength)
    for wavelength in wavelengths
])
em_dcounts_sampler_val = np.array([
    em_dcounts_sampler(test_energy, wavelength)
    for wavelength in wavelengths
])
em_long_val = np.array([
    em_long(test_energy, z)
    for z in zgrid
])
em_angles_val = np.array([
    em_angles(angle, 1.33)
    for angle in angle_grid
])

In [ ]:
# How to calculate the total counts
print(np.trapz(em_dcounts_val, wavelengths))
print(np.trapz(em_dcounts_internal, wavelengths))

In [ ]:
# Plotting some EM stuff
# Photon counts
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(wavelengths, em_dcounts_val, linewidth=lw, color='k')
ax1.step(wavelengths, em_dcounts_sampler_val, linewidth=lw, color='r')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Wavelength\;[nm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Differential\;Counts$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;e^-$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_EM_Wavelength_JAX.png",
               bbox_inches='tight', dpi=export_dpi)
# ----------------------------------------------
# ----------------------------------------------
# Long
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(zgrid, em_long_val / np.trapz(em_long_val, x=zgrid),
         linewidth=lw, color='k')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('log')
ax1.set_xlabel(r'$Distance\;[cm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Long.\;Distro.$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;e^-$", fontsize=fontsize)
ax1.set_xlim(0, 2500)
ax1.set_ylim(1e-13, 5e-3)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_EM_Long_JAX.png",
               bbox_inches='tight', dpi=export_dpi)
# ----------------------------------------------
# ----------------------------------------------
# Angles
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(angle_grid, em_angles_val / np.trapz(em_angles_val, x=angle_grid),
         linewidth=lw, color='k')
ax1.plot(angle_grid, em_angles_val / np.trapz(em_angles_val, x=angle_grid),
         linewidth=lw, color='r', ls='--')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Emission\;Angle$', fontsize=fontsize)
ax1.set_ylabel(r'$PDF$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;e^-$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_EM_Angle_JAX.png",
               bbox_inches='tight', dpi=export_dpi)

In [ ]:
# Hadron Cascade
had_dcounts, had_dcounts_sampler, had_em_frac, had_em_frac_sample, had_long, had_angles = (
    fennel_instance.hadron_yields(
        energy=test_energy,
        particle=211,
        function=True
    )
)
# Using the internal value fetcher as a comparison
had_dcounts_internal, had_dcounts_sampler_internal, had_em_frac_internal, had_em_frac_sample_internal, had_long_internal, had_angles_internal = (
    fennel_instance.hadron_yields(
        energy=test_energy,
        particle=-211,
        function=False
    )
)

In [ ]:
# Wrap in list comprehension for multiple values
had_dcounts_val = np.array([
    had_dcounts(test_energy, wavelength)
    for wavelength in wavelengths
])
had_dcounts_sampler_val = np.array([
    had_dcounts_sampler(test_energy, wavelength)
    for wavelength in wavelengths
])
had_em_frac_mean = had_em_frac(test_energy)
had_em_frac_samples = had_em_frac_sample(test_energy)
had_long_val = np.array([
    had_long(test_energy, z)
    for z in zgrid
])
had_angles_val = np.array([
    had_angles(test_energy, angle, 1.33)
    for angle in angle_grid
])

In [ ]:
# How to calculate the total counts
print(np.trapz(had_dcounts_val, wavelengths))
print(np.trapz(had_dcounts_internal, wavelengths))

In [ ]:
# Plotting some hadron stuff
# Photon counts
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(wavelengths, had_dcounts_val, linewidth=lw, color='k')
ax1.step(wavelengths, had_dcounts_sampler_val, linewidth=lw, color='r', ls='-')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Wavelength\;[nm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Differential\;Counts$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;\pi^+$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(
    PICS + "Single_Had_Wavelength_JAX.png",
    bbox_inches='tight', dpi=export_dpi)
# ----------------------------------------------
# ----------------------------------------------
# Long
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(zgrid, had_long_val / np.trapz(had_long_val, x=zgrid),
         linewidth=lw, color='k')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('log')
ax1.set_xlabel(r'$Distance\;[cm]$', fontsize=fontsize)
ax1.set_ylabel(r'$Long.\;Distro.$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;\pi^+$", fontsize=fontsize)
ax1.set_xlim(0, 2500)
ax1.set_ylim(1e-10, 5e-3)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Single_Had_Long_JAX.png",
               bbox_inches='tight', dpi=export_dpi)
# ----------------------------------------------
# ----------------------------------------------
# Angles
figure, (ax1) = plt.subplots(
    1, 1,
    figsize=(std_size, std_size * 6. / 8.),
)
ax1.plot(angle_grid, had_angles_val / np.trapz(had_angles_val, x=angle_grid, axis=0),
         linewidth=lw, color='k')
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$Emission\;Angle$', fontsize=fontsize)
ax1.set_ylabel(r'$PDF$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_title("$10\;TeV\;\pi^+$", fontsize=fontsize)
# ax1.set_xlim(500, 600)
# ax1.set_ylim(1e-3, 1e2)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(
    PICS + "Single_Had_Angle_JAX.png",
    bbox_inches='tight', dpi=export_dpi)

In [ ]:
# Example of the auto_yield function
# Here we're fetching the counts
# All other functions/values are also constructed
# Muon
auto_13 = (
    fennel_instance.auto_yields(
        energy=test_energy,
        particle=13,
        function=True
    )
)
auto_counts_13 = np.array([
    auto_13[0](test_energy, wavelength)
    for wavelength in wavelengths
])
# Electron
auto_11 = (
    fennel_instance.auto_yields(
        energy=test_energy,
        particle=11,
        function=True
    )
)
auto_counts_11 = np.array([
    auto_11[0](test_energy, wavelength)
    for wavelength in wavelengths
])
# Pion
auto_211 = (
    fennel_instance.auto_yields(
        energy=test_energy,
        particle=211,
        function=True
    )
)
auto_counts_211 = np.array([
    auto_211[0](test_energy, wavelength)
    for wavelength in wavelengths
])
# The different totals
print(np.trapz(auto_counts_13, wavelengths))
print(np.trapz(auto_counts_11, wavelengths))
print(np.trapz(auto_counts_211, wavelengths))

In [ ]:
# Plotting some hadron stuff (BETA!)
# Muon Counts
primary_grid = np.array([1e2, 1e3, 1e4])
seconday_grid = np.arange(1., 10**3.5, step=1.)
particles_to_check = [
    fennel_instance._particles[211],
    # fennel_instance._particles[130],
    # fennel_instance._particles[2212]
]
line_colors = ["b", "g", "k"]
line_styles = ["-", "--", ":"]
# Fetching desired results
distro_lib = {}
for particle in particles_to_check:
    distro_lib[particle._name] = (
        fennel_instance._hadron_cascade._muon_production_fetcher(
            primary_grid, seconday_grid, particle
        )
    )
figure, (ax1) = plt.subplots(1, 1, figsize=(std_size, std_size * 6. / 8.), sharex=True)
# Plotting
for id_prim, primary in enumerate(primary_grid):
    for id_part, particle in enumerate(particles_to_check):
        ax1.plot(
            np.log10(seconday_grid),
            distro_lib[particle._name][id_prim] / primary /
            np.trapz(distro_lib[particle._name][id_prim],
            x=seconday_grid),
            linewidth=lw,
            color=line_colors[id_part], ls=line_styles[id_prim]
        )
# ---------------------------------------------
# Plotting options
ax1.set_xscale('linear')
ax1.set_yscale('log')
ax1.set_xlabel(r'$log_{10}\left(\frac{E}{GeV}\right)$', fontsize=fontsize)
ax1.set_ylabel(r'$\frac{N_\mu}{E_{prim}N_{tot}}\left(E_\mu > E\right)$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.5, +1.27),
                  ncol=3, fontsize=fontsize, handlelength=h_length,
                  fancybox=True, frameon=False)
ax1.add_artist(lgd1)
ax1.set_xlim(-0.5, 2.)
ax1.set_ylim(1e-7, 1e-1)
for id_prim, prim in enumerate(primary_grid):
    max_val = max(
        distro_lib[particles_to_check[0]._name][id_prim] / prim /
        np.trapz(distro_lib[particles_to_check[0]._name][id_prim], x=seconday_grid))
    ax1.text(-0.2, max_val, "%.e GeV" % prim, fontsize=fontsize/2)
ax1.set_title("$\mu\;Production$", fontsize=fontsize)
plt.show()
# ---------------------------------------------
# Storing the image
figure.savefig(PICS + "Muon_counts.png",
               bbox_inches='tight', dpi=export_dpi)